<img src="https://github.com/neoscalc/IntersecT/blob/main/src/pyIntersecT/Logo-IntersecT.png?raw=true" width="200">

# **IntersecT**
## Quantitative Isopleth Thermobarometry from Thermodynamic Models
---

## Introduction

Phase diagrams calculated using forward thermodynamic modelling allow users to determine the pressure (*P*) and temperature (*T*) conditions of mineral assemblage formation for a given bulk composition (*X*). In the last decade, several software solutions have provided quantitative approaches where optimal *P–T* conditions can be calculated from a statistical point of view, resulting in more accurate interpretation of forward model results (e.g., the quality factor implemented in Bingo-Antidote; Duesterhoeft & Lanari, 2020, https://doi.org/10.1111/jmg.12538).

Here the quality factor of the composition (hereby **Qcmp**) as described in Duesterhoeft & Lanari (2020) is applied to thermodynamic model results from both **Perple_X** (WERAMI output) and **MAGEMin**. The Qcmp allows the propagation of uncertainties in the measured mineral composition. In addition, IntersecT applies a reduced chi-squared statistics to assess the weight of the considered phases. This allows for the down-weighting of outlier data, whether due to model problems or incorrect assumptions, including disequilibrium features.

This implementation uses standardized mineral nomenclature following Warr (2021, https://doi.org/10.1180/mgm.2021.43) and automatically handles phase name resolution from software-specific abbreviations. Solid solution phases are discriminated where appropriate based on compositional criteria.

##### **Requirements**
The following Python packages are required. Installation of pyIntersecT through pip will automatically install all necessary dependencies.
- Python 3.9 or higher
- matplotlib
- pandas
- tkinter
- **pyIntersecT**

## Install and import the required libraries

In [ ]:
# Istall the required packages in a local environment
# Execute this cell once if the package is not already installed, then ignore
%pip install pyIntersecT

In [ ]:
# Import IntersecT
from pyIntersecT import IntersecT

# Create a Quality Factor Analysis
InT = IntersecT.QualityFactorAnalysis.from_default_symbols()

## Output Directory Selection

Select a directory where to store the output. The system creates a log file (log_IntersecT.txt) recording all calculations for documentation. All visualizations are saved as PDF files with vector graphics.

In [ ]:
# Set the output directory
InT.set_output_directory()

## Input Files

### Model Output File

Select your model output file from either **Perple_X** (WERAMI option 36, properties of the system and its phases, as .phm file) or **MAGEMin** (.csv file saved from Diagram>Informations>Save all). The system automatically detects file format and coordinate structure.

N.B. Because it's required to propagate the uncertainty of the measured composition, the extracted composition should be in **atoms per formula unit (a.p.f.u.)**.

Therefore:

If using Perple_X -> ensure that "mol" is used in the perplex_option file in composition_system and composition_phase

If using MAGEMin -> ensure you have the latest version installed

In [ ]:
# Load the model output file from Perple_X or MAGEMin
InT.load_model_output()

## Coordinate Configuration

The system suggests appropriate coordinate variables based on your data structure. Common configurations include temperature-pressure diagrams (*T-P*), composition-pressure sections (*X-P*), or temperature-composition diagrams (*T-X*). 

You can accept the suggestions or specify coordinates manually if needed.

In [ ]:
# auto-detect and suggest appropriate coordinates for plotting
x_coord, y_coord = InT.suggest_plot_coordinates()
print(f"Suggested coordinates: {x_coord} (x-axis) and {y_coord} (y-axis)")

# accept the suggestions
InT.set_plot_coordinates(x_coord, y_coord)

# alternative: manually specify coordinates if needed
# InT.set_plot_coordinates("T[°C]", "P[kbar]")  # for T-P diagram
# InT.set_plot_coordinates("X[0.0-1.0]", "P[kbar]")  # for X-P diagram

### Measured Composition File

Select a second tab-delimited **.txt** input file with the **a.p.f.u. measured compositions** that match the model output, the **name** of the considered phases (using Warr 2021 nomenclature), and the **color scheme** you wish to apply. Phase names should follow the standard abbreviations from Warr (2021): Grt (garnet), Bt (biotite), Chl (chlorite), etc.

**Row 1 - Element identifiers:** Specify each element using the format Phase_Element. For example, garnet compositions would be listed as Grt_Mg, Grt_Ca, Grt_Fe, Grt_Mn. If uncertain on the naming, search for the output model naming in the mineral_symbols.toml and use the corresponding abbreviation.

**Row 2 - Observed values:** Measured compositional values expressed in atoms per formula unit (a.p.f.u.) corresponding to each element specified in row 1.

**Row 3 - Analytical uncertainties:** Either numerical uncertainty values for each measurement or a dash (-) to trigger automatic uncertainty calculation based on the analytical technique specified in row 4.

**Row 4 - Analysis type:** Specify the analytical technique employed. Accepted values are EDS, WDS map, or WDS spot. This information determines the appropriate uncertainty model if explicit uncertainties are not provided.

**Row 5:** Leave blank.

**Row 6 - Color scheme:** Specify the matplotlib colormap for visualization. Recommended options include viridis, plasma, inferno, magma, cividis, jet or any other valid matplotlib colormap name.

In [ ]:
# Import the measured compositions
InT.import_analytical_compo()

## Build Data Table

This step integrates model predictions with observed compositions, constructing the internal data structures required for quality factor calculations. The system resolves phase names from software-specific abbreviations to Warr (2021) nomenclature and applies compositional discrimination for solid solutions.

In [ ]:
# Construct the integrated data table
InT.build_intersect_table()

# Show a sample of the data to verify proper phase-element matching
display(InT.phase_data.head())

## Quality Factor and Reduced Chi-Squared Calculations

This part of the code **calculates the Qcmp**. It then integrates the **reduced chi-squared** statistic to calculate a **weighted Qcmp** based on the best reduced chi-squared value for each phase.

The quality factor (Qcmp) quantifies the agreement between measured and modeled compositions while incorporating analytical uncertainties. Values range from 0 to 100%. The calculation follows Duesterhoeft and Lanari (2020).

The reduced chi-squared statistic provides an independent evaluation of model-data fit. Values near 1.0 indicate that observed discrepancies are consistent with analytical uncertainty alone, while values substantially greater than 1.0 suggest systematic misfit potentially arising from disequilibrium, analytical bias, or thermodynamic model limitations. This statistic weights the contribution of different phases to the total quality factor, systematically down-weighting phases that exhibit poor statistical agreement.

### Element-Specific Quality Factors

This first part calculates the Qcmp for each **element** extracted from the model output.

In [ ]:
# Calculate quality factors for each element
# This generates separate plots for each phase-element combination
# Output: PDF files named "Qcmp_PhaseName_Element.pdf" in the output directory
InT.Qcmp_elem()

### Phase-Specific Quality Factors

This part calculates the Qcmp for each **phase** (combining all elements).

The phase-specific quality factor provides a comprehensive assessment of how well the model reproduces the complete composition of each phase.

In [ ]:
# Calculate quality factors for each phase
# This combines all elements within each phase and generates phase-specific plots
# Output: PDF files named "Qcmp_PhaseName.pdf" in the output directory
Qcmp_phase_tot = InT.Qcmp_phase()

### Phase-Specific Reduced Chi-Squared Statistics

This calculates the **reduced chi-squared** for each **phase**.

For phases with more than two measured elements, the reduced chi-squared is calculated by dividing the chi-squared statistic by degrees of freedom (number of elements minus one). For phases with only one or two measured elements, the chi-squared value itself is plotted and labeled accordingly, as reduction by degrees of freedom would be statistically inappropriate with insufficient constraints.

In [ ]:
# Calculate reduced chi-squared statistics for each phase
# Output: PDF files named "redχ2_PhaseName.pdf" or "χ2_PhaseName.pdf" in the output directory
redchi2_phase = InT.redchi2_phase()

### Total Reduced Chi-Squared for Complete Assemblage

This calculates the **reduced chi-squared** for the **overall dataset**.

By combining all measured elements across all phases, this metric provides a unified statistical assessment of how well the model reproduces the complete observed mineral chemistry at each x-y condition.

In [ ]:
# Calculate reduced chi-squared for the overall dataset
# Output: PDF file named "redχ2_tot.pdf" in the output directory
redchi2_allphases = InT.redchi2_tot()

### Unweighted Total Quality Factor

This part calculates an **unweighted** Qcmp for the **overall dataset** (combining all phases).

In this calculation, all phases receive equal weight regardless of their individual statistical reliability.

This approach is appropriate when all phases are believed to represent equilibrium conditions with comparable reliability, or when reduced chi-squared values are similar across phases. The resulting map identifies x-y conditions that optimally satisfy all compositional constraints simultaneously under the assumption of equal phase importance.

In [ ]:
# Calculate the unweighted total quality factor
# All phases contribute equally regardless of statistical reliability
# Output: PDF file named "Unweighted_Qcmp_tot.pdf" in the output directory
Qcmp_allphases = InT.Qcmp_tot(Qcmp_phase_tot, redchi2_phase)

### Weighted Total Quality Factor

Integrating the reduced chi-squared statistic, this calculates a **weighted Qcmp** for the **overall dataset**. Phases with better statistical agreement receive higher weight, allowing down-weighting of outlier phases while preserving information from all measurements.

In this calculation, each phase's contribution to the total quality factor is weighted by the inverse of its minimum reduced chi-squared value, thereby assigning greater influence to phases demonstrating better statistical agreement between model and observations.

In [ ]:
# Calculate the weighted total quality factor
# Phase contributions are weighted by statistical reliability (inverse reduced chi-squared)
# Output: PDF file named "Weighted_Qcmp_tot.pdf" in the output directory
Qcmp_allphases_weight = InT.Qcmp_tot_weight(Qcmp_phase_tot, redchi2_phase)

## Results

All output files (PDF plots and log_IntersecT.txt) are saved in your selected directory.

For theoretical background and interpretation guidance, consult Duesterhoeft and Lanari (2020) regarding the quality factor approach, and Warr (2021) for mineral nomenclature conventions employed in phase identification. Careful examination of both weighted and unweighted results is recommended, as substantial differences between them may indicate disequilibrium conditions or variable data quality across the phase assemblage. The weighted result should generally be preferred as it accounts for the statistical reliability of different phases, but significant discrepancies between weighted and unweighted estimates warrant careful scrutiny of individual phase matches and potential sources of systematic error.